<a href="https://colab.research.google.com/github/rwcitek/anl-nerde/blob/rwc/nerde_argonne_api_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NERDE API via Python



NERDE ( National Economic Resilience Data Explorer )



- https://www.anl.gov/dis/national-economic-resilience-data-explorer-nerde
- https://nerde.anl.gov/
- [Introduction to Nerde]( https://youtu.be/FCzSLtNCG5U?feature=shared ) on YouTube
- [API Docs]( https://nerde.anl.gov/resources/api-documentation )
  - [account registration/sign in]( https://nerde.anl.gov/auth/login )


Other resources:
- [Federal Information Processing Standards (FIPS) ]( https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards )

## Setting up the Request


### Obtaining an API Token


In [1]:
# Get u/p credentials from Colab Vault
from google.colab import userdata

nerde_username = userdata.get('nerde_username')
nerde_password = userdata.get('nerde_password')


In [2]:
import requests
import json
import pandas as pd


In [3]:
call = "https://nerde.anl.gov"

# Pull API Token
response = requests.post(
  url=f"{call}/api/auth/login",
  json={
    "userName": nerde_username,  # Remember to replace your username
    "password": nerde_password,  # Remember to replace your password
    "rememberMe": True
  }
)
response


<Response [200]>

In [4]:
token = response.json()

key = token['result']['token'] # Your unique key
f"{key[:20]}...{key[-20:]}"

'eyJhbGciOiJIUzI1NiIs...hPJkyIgymqQUI99jwG-4'

### Establishing parameters




In [5]:
# FIPS codes
my_fips = '24011,24013,24015,24017,24019,24021,24023,24025,24027,24029,24031,24033,24035,24039,24037,24041,24043,24045,24047'
# Start and End dates
my_start = '2020-01-01'
my_end = '2023-01-01'


### Defining the function to make the data request




In [6]:
def NERDE_API(section_url, fips="24011", startdate='2020-01-01', enddate='2023-01-01'):
  response = requests.get(
    url=f"{call}{section_url}?county_fips={fips}&start_date={startdate}&end_date={enddate}",
    headers={"Authorization": f"Bearer {key}"}
  ).json()

  # Each county will populate a list, this loops through each called county to parse results
  results_summary_digest = [ result for result in response['result'] ]

  return results_summary_digest


## Getting County Level Data



### Summary Data



In [7]:
# Create the variable to point to the correct dataset you'd like to pull data from
section_url = "/api/v1/county/explorer/summary"


In [8]:
# API call
summary_digest = NERDE_API(section_url, my_fips, my_start, my_end)


In [9]:
type(summary_digest)

list

In [10]:
# Create a Pandas DataFrame from summary_digest
main_summary_df = pd.json_normalize(summary_digest)


In [11]:
main_summary_df.shape

(19, 14)

In [12]:
main_summary_df

,county_fips,county,state,total_population,census_per_capita_income,designated_coal_community,adjacent_tribal_land,nuclear_power_plant_present,persistent_poverty_designation,pct_urban_2020,pct_rural_2020,bea_per_capita_income,local_24month_unemployment_rate_series,edci_indicators
0,24011,Caroline County,Maryland,33406,33898,NaN,None,None,No,0.150452,0.849548,56447,"[{'value': 0.0372016509117358, 'timestamp': '2...","[{'score': -1.69499694620526, 'capacity': 'Inf..."
1,24013,Carroll County,Maryland,174318,51465,NaN,None,None,No,0.577612,0.422388,74895,"[{'value': 0.0296015539496292, 'timestamp': '2...","[{'score': -1.63513699632672, 'capacity': 'Inf..."
2,24015,Cecil County,Maryland,104366,42560,NaN,None,None,No,0.516520,0.483480,59373,"[{'value': 0.0397263205400315, 'timestamp': '2...","[{'score': -1.80146427534243, 'capacity': 'Inf..."
3,24017,Charles County,Maryland,168710,50540,1.0,None,None,No,0.715851,0.284149,66642,"[{'value': 0.0352469640229852, 'timestamp': '2...","[{'score': -3, 'capacity': 'Infrastructure', '..."
4,24019,Dorchester County,Maryland,32612,34780,NaN,None,None,No,0.460422,0.539578,57060,"[{'value': 0.0424337032877077, 'timestamp': '2...","[{'score': -1.23962080337455, 'capacity': 'Hum..."
5,24021,Frederick County,Maryland,280341,53316,1.0,None,None,No,0.724588,0.275412,76664,"[{'value': 0.0290808153505861, 'timestamp': '2...","[{'score': -1.62025812191036, 'capacity': 'Inf..."
6,24023,Garrett County,Maryland,28713,42328,1.0,None,None,No,0.157884,0.842116,57525,"[{'value': 0.0427417766397479, 'timestamp': '2...","[{'score': -2.01058199858782, 'capacity': 'Inf..."
7,24025,Harford County,Maryland,262509,50861,NaN,None,None,No,0.790713,0.209287,71196,"[{'value': 0.0331162769451329, 'timestamp': '2...","[{'score': -3, 'capacity': 'Infrastructure', '..."
8,24027,Howard County,Maryland,333916,65860,NaN,None,None,No,0.881225,0.118775,95529,"[{'value': 0.0285709624218793, 'timestamp': '2...","[{'score': -2.09380757210108, 'capacity': 'Ind..."
9,24029,Kent County,Maryland,19265,45817,NaN,None,None,No,0.303730,0.696270,77025,"[{'value': 0.0411409553474282, 'timestamp': '2...","[{'score': -1.13022441291406, 'capacity': 'Ins..."


In [13]:
main_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   county_fips                             19 non-null     object 
 1   county                                  19 non-null     object 
 2   state                                   19 non-null     object 
 3   total_population                        19 non-null     int64  
 4   census_per_capita_income                19 non-null     int64  
 5   designated_coal_community               7 non-null      float64
 6   adjacent_tribal_land                    0 non-null      object 
 7   nuclear_power_plant_present             0 non-null      object 
 8   persistent_poverty_designation          19 non-null     object 
 9   pct_urban_2020                          19 non-null     float64
 10  pct_rural_2020                          19 non-null     float64


In [14]:
main_summary_df["designated_coal_community"].value_counts( dropna=False)

,count
designated_coal_community,
NaN,12
1.0,7


In [15]:
main_summary_df["nuclear_power_plant_present"].value_counts( dropna=False)

,count
nuclear_power_plant_present,
None,19


Our returned dataframe contains binary column data that has been returned as a floating number as well as several missing values. Let's clean that up...



### Cleaning the Dataframe



In [16]:
def update_column_values(df, column_name):
  # Replace 1.0 with 'Yes' and NaN with 'No'
  df[column_name] = df[column_name].apply(lambda x: 'Yes' if x == 1.0 else ( 'No' if pd.isna(x) else x ))

  # Set column to string type
  df[column_name] = df[column_name].astype(str)

  return df


In [17]:
main_summary_df = update_column_values(main_summary_df, 'designated_coal_community')
main_summary_df = update_column_values(main_summary_df, 'nuclear_power_plant_present')


In [18]:
main_summary_df["designated_coal_community"].value_counts( dropna=False)

,count
designated_coal_community,
No,12
Yes,7


In [19]:
main_summary_df["nuclear_power_plant_present"].value_counts( dropna=False)

,count
nuclear_power_plant_present,
No,19


### Separate Unemploment data and EDCI Indicators and flattened nested dictionaries




In [20]:
# Create function to un-nest columns
def flatten_column_data(df, list_column, county_column='county', code_column='county_fips'):
  flattened_rows = []

  for _, row in df.iterrows():
    county = row[county_column]
    code = row[code_column]
    for entry in row[list_column]:
      flattened_entry = {
        'County': county,
        'Code': code
      }
      # Add all key-value pairs from the dictionary to the flattened entry
      for key, value in entry.items():
        flattened_entry[key] = value
      flattened_rows.append(flattened_entry)

  return pd.DataFrame(flattened_rows)


In [21]:
# Create function to un-nest columns
def flatten_column_data_2(df, list_column):
  # explode list_column
  df_1 = df.explode(list_column)

  # convert dictionaries to fields
  df_2 = df_1[list_column].apply(pd.Series)

  # join data frames on their index and drop original list column
  return df.join(df_2).drop(columns = list_column)


In [22]:
sorted(main_summary_df.columns)

['adjacent_tribal_land',
 'bea_per_capita_income',
 'census_per_capita_income',
 'county',
 'county_fips',
 'designated_coal_community',
 'edci_indicators',
 'local_24month_unemployment_rate_series',
 'nuclear_power_plant_present',
 'pct_rural_2020',
 'pct_urban_2020',
 'persistent_poverty_designation',
 'state',
 'total_population']

In [23]:
# Unemployment Data
unemployment_edci_data = main_summary_df[['county', 'county_fips', 'local_24month_unemployment_rate_series']]


In [24]:
unemployment_edci_data.shape

(19, 3)

In [25]:
unemployment_edci_data[:5]

,county,county_fips,local_24month_unemployment_rate_series
0,Caroline County,24011,"[{'value': 0.0372016509117358, 'timestamp': '2..."
1,Carroll County,24013,"[{'value': 0.0296015539496292, 'timestamp': '2..."
2,Cecil County,24015,"[{'value': 0.0397263205400315, 'timestamp': '2..."
3,Charles County,24017,"[{'value': 0.0352469640229852, 'timestamp': '2..."
4,Dorchester County,24019,"[{'value': 0.0424337032877077, 'timestamp': '2..."


In [26]:
# Create a new unemployment DataFrame with flattened data
flattened_unemployment_data = flatten_column_data(unemployment_edci_data, 'local_24month_unemployment_rate_series')


In [27]:
flattened_unemployment_data.shape

(703, 4)

In [28]:
flattened_unemployment_data[:5]

,County,Code,value,timestamp
0,Caroline County,24011,0.037202,2020-02-01
1,Caroline County,24011,0.037232,2020-03-01
2,Caroline County,24011,0.037641,2020-01-01
3,Caroline County,24011,0.037756,2023-01-01
4,Caroline County,24011,0.039085,2020-04-01


In [29]:
flatten_column_data_2(unemployment_edci_data, 'local_24month_unemployment_rate_series')[:5]

,county,county_fips,value,timestamp
0,Caroline County,24011,0.037202,2020-02-01
0,Caroline County,24011,0.037232,2020-03-01
0,Caroline County,24011,0.037641,2020-01-01
0,Caroline County,24011,0.037756,2023-01-01
0,Caroline County,24011,0.039085,2020-04-01


In [30]:
# EDCI Indicators Data
edci_data = main_summary_df[['county', 'county_fips', 'edci_indicators']]


In [31]:
# Create a new EDCI DataFrame with flattened data
flattened_edci_data = flatten_column_data(edci_data, 'edci_indicators')


In [32]:
flattened_edci_data.shape

(1121, 7)

In [33]:
flattened_edci_data[:5]

,County,Code,score,capacity,indicator,percentile,directionality
0,Caroline County,24011,-1.694997,Infrastructure,perc_selfsupply,0.075941,negative
1,Caroline County,24011,-1.211996,Infrastructure,perc_stream_impaired,0.156863,negative
2,Caroline County,24011,-1.130224,Institutions and Partnerships,local_govt_employees_per_capita,0.032991,positive
3,Caroline County,24011,-0.798065,Industry Composition,estabs_exit_rate,0.203859,negative
4,Caroline County,24011,-0.636447,Human Capital,pct_gross_rent_as_pct_of_hhold_income_above_35pct,0.245876,negative


In [34]:
flatten_column_data_2(edci_data, 'edci_indicators').shape

(1121, 7)

In [35]:
flatten_column_data_2(edci_data, 'edci_indicators')[:5]

,county,county_fips,score,capacity,indicator,percentile,directionality
0,Caroline County,24011,-1.694997,Infrastructure,perc_selfsupply,0.075941,negative
0,Caroline County,24011,-1.211996,Infrastructure,perc_stream_impaired,0.156863,negative
0,Caroline County,24011,-1.130224,Institutions and Partnerships,local_govt_employees_per_capita,0.032991,positive
0,Caroline County,24011,-0.798065,Industry Composition,estabs_exit_rate,0.203859,negative
0,Caroline County,24011,-0.636447,Human Capital,pct_gross_rent_as_pct_of_hhold_income_above_35pct,0.245876,negative


> **Tip**
>
> If you run into `KeyError: 'result'` the API Key has expired and you should re-run the section [Obtaining an API Token](#obtaining-an-api-token)



### Demographics



In [36]:
# Demographics API end point
demographics_url="/api/v1/county/explorer/demographics"


In [37]:
# Calling the API
demographics_digest = NERDE_API(demographics_url, my_fips, my_start, my_end)


In [38]:
# Turn response into Dataframe
demographics_df = pd.json_normalize(demographics_digest)


In [39]:
# Annual Demographics Dataframe
demographics_nested_df = demographics_df[['county', 'county_fips', 'annual_demographics']]


In [40]:
# Annual Demographics Un-nested
demographics_annnual_df = flatten_column_data(demographics_nested_df, 'annual_demographics')


### Housing


In [41]:
#Housing API end point
housing_url="/api/v1/county/explorer/housing"


In [42]:
# Calling the API
housing_digest = NERDE_API(housing_url, my_fips, my_start, my_end)


In [43]:
# Turn response into Dataframe
housing_df = pd.json_normalize(housing_digest)


### Workforce



In [44]:
# Workforce API end point
workforce_url = "/api/v1/county/explorer/workforce"


In [45]:
# Calling the API
workforce_digest = NERDE_API(workforce_url, my_fips, my_start, my_end)


In [46]:
# Turn response into Dataframe
workforce_df = pd.json_normalize(workforce_digest)


In [47]:
# Monthly Economic Statistics Dataframe
workforce_monthly_stats = workforce_df[['county', 'county_fips', 'monthly_economic_stats']]


In [48]:
# Monthly Economic Statistics Un-nested
workforce_monthly_stats_flattened = flatten_column_data(workforce_monthly_stats, 'monthly_economic_stats')


### Local Economy



In [49]:
# Local Economy URL
localeconomy_url = "/api/v1/county/explorer/localeconomy"


In [50]:
# Calling the API
localeconomy_digest = NERDE_API(localeconomy_url, my_fips, my_start, my_end)


In [51]:
# Turn response into Dataframe
localeconomy_df = pd.json_normalize(localeconomy_digest)


In [52]:
# Industry Employment Dataframe
localeconomy_industry_employment_series = localeconomy_df[['county', 'county_fips', 'state', 'industry_employment_series']]


In [53]:
# Industry Employment Un-nested
localeconomy_industry_employment_series_flattened = flatten_column_data(localeconomy_industry_employment_series, 'industry_employment_series')


In [54]:
# Industry GDP Dataframe
localeconomy_industry_gdp_series = localeconomy_df[['county', 'county_fips', 'state', 'industry_gdp_series']]


In [55]:
# Industry GDP Un-nested
localeconomy_industry_gdp_series_flattened = flatten_column_data(localeconomy_industry_gdp_series, 'industry_gdp_series')


### Industry Trends



In [56]:
# Industry Trends URL
industrytrends_url="/api/v1/county/explorer/industrytrends"


In [57]:
# Calling the API
industry_digest = NERDE_API(industrytrends_url, my_fips, my_start, my_end)


In [58]:
# Turn response into Dataframe
industry_df = pd.json_normalize(industry_digest)


In [59]:
# Industry Employment Dataframe
industry_location_quotient_series_df = industry_df[['county', 'county_fips', 'location_quotient_series']]


In [60]:
# Industry Employment Un-nested
industry_location_quotient_series_df_flattened = flatten_column_data(industry_location_quotient_series_df, 'location_quotient_series')


### Risk And Resilience


In [61]:
# Rist and Resilience URL
risk_url="/api/v1/county/explorer/riskresilience"


In [62]:
# Calling the API
risk_digest = NERDE_API(risk_url, my_fips, my_start, my_end)


In [63]:
# Turn response into Dataframe
risk_df = pd.json_normalize(risk_digest)


In [64]:
# Rename Column
risk_df.rename(columns={"geo_fips": "county_fips"}, inplace=True)


In [65]:
# Risk FEMA Disaster Declarations Dataframe
risk_fema_disaster_dec_df = risk_df[['county', 'county_fips', 'fema_disaster_declarations']]


In [66]:
# Risk FEMA Disaster Declarations Un-nested
risk_fema_disaster_dec_df_flatten = flatten_column_data(risk_fema_disaster_dec_df, 'fema_disaster_declarations')
